**IMPORTAR LIBRERIAS**

In [12]:
import os
import glob
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.cm as cm

**DEFINIR RUTAS DEL DATASET**

In [13]:
# Dataset ya cargado en kaggle (agregado con Add Data)
train_path ="/kaggle/input/covid19-image-dataset/Covid19-dataset/train"

# Obtener etiquetas disponibles (COVID, Normal, Viral Pneumonia)
labels = os.listdir(train_path)
print(f"Clases encontradas: {labels}")

Clases encontradas: ['Normal', 'Viral Pneumonia', 'Covid']


**CARGAR Y PREPARAR IMAGENES**

In [14]:
X = []
y = []

for label_int, label_string in enumerate(labels):
    path = os.path.join(train_path, label_string)
    image_files = glob.glob(path + "/*.jpg") + glob.glob(path + "/*.jpeg") + glob.glob(path + "/*.png")

    for img_file in image_files:
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224,224))
        X.append(img)
        y.append(label_int)

X = np.array(X)
y = np.array(y)

# Preprocesar imagenes como espera MobileNetV2
X = preprocess_input(X)

print(f"Forma final de X: {X.shape}")

Forma final de X: (251, 224, 224, 3)


**DIVIDIR DATA EN ENTRENAMIENTO Y VAIDACIÓN**

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Conjunto de Entrenamiento: {X_train.shape}")
print(f"Conjunto de Validación: {X_val.shape}")

Conjunto de Entrenamiento: (200, 224, 224, 3)
Conjunto de Validación: (51, 224, 224, 3)


**CREAR MODELO TRANSFER LEARNING**

In [17]:
# Cargar MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freezear las capas base
base_model.trainable = False

# Input
inputs = tf.keras.Input(shape=(224,224,3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(labels), activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

# Compilar Modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ver resumen
model.summary()

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5: None -- [Errno -3] Temporary failure in name resolution

**ENTRENAMIENTO DEL MODELO**

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    eppchs=100
    batch-size=32
)